---
<h1 align="center"><strong>Cálculo de dependencia cruzada para todos los países</strong></h1>
<h4 align="center"><strong>Manuel Alejandro Hidalgo y Jorge Díaz Lanchas</strong></h4>
<h4 align="center"><strong>Fundación Real Instituto Elcano</strong></h4>

---

# Esquema para Índice de Seguridad Económica - Real Instituto Elcano

## 1. Introducción y Marco Conceptual

- **Objetivo**: Desarrollar un índice que cuantifique la seguridad económica de los países
- **Definición**: La seguridad económica como la capacidad de un país para resistir disrupciones en sus cadenas de suministro y comercio internacional
- **Relevancia**: Contexto actual de fragmentación geoeconómica y tensiones comerciales

## 2. Metodología

### 2.1 Fuentes de Datos
- Base de datos International Trade and Production Database (ITP)
- Datos comerciales bilaterales por industria (año 2019)
- Otros indicadores macroeconómicos complementarios

### 2.2 Procesamiento de Datos
```python
# Procesamiento y carga de datos ITP
itp2019, codigos_countries = procesar_datos_itp()
```

### 2.3 Creación de Matrices de Comercio
```python
# Generación de matrices bilaterales por industria
matrices_comercio = crear_matriz_comercio(data.groupby('industry_descr'), codigos_paises)
```

### 2.4 Limpieza y Filtrado
```python
# Eliminar relaciones comerciales insignificantes
mat_clean = eliminar_filas_columnas_cero(mat, threshold_pct=0.05)
```

### 2.5 Cálculo de Dependencias Económicas
```python
# Cálculo de dependencias directas e indirectas
results = analyze_dependencies(X, country_names)
```

## 3. Componentes del Índice

### 3.1 Dependencia Directa
- Medición de la dependencia inmediata entre países
- Fórmula: $D_{ij} = \frac{X_{ji}}{∑_k X_{ki}}$, donde $X_{ji}$ es el comercio del país j al país i

### 3.2 Dependencia Indirecta
- Medición de dependencias a través de cadenas de suministro
- Incorporación de países intermediarios en las relaciones comerciales
- Análisis de caminos hasta longitud 5

In [15]:
# Limitar hilos de BLAS antes de importar NumPy (evita oversubscription cuando paralelicemos)
import os as _os
_os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
_os.environ.setdefault("MKL_NUM_THREADS", "1")
_os.environ.setdefault("OMP_NUM_THREADS", "1")
_os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

import os
import gzip
import pandas as pd
import numpy as np
from numpy.linalg import inv
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import combinations
import torch

# Añadimos soporte esparso para futuros cálculos eficientes
from scipy import sparse  # <- nuevo

import dask.dataframe as dd
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict
import multiprocessing
from joblib import Parallel, delayed

# (Opcional) detección segura de CuPy sin importarlo directamente (evita aviso del linter)
import importlib.util as _importlib_util
_cp_spec = _importlib_util.find_spec("cupy")
cp = None  # placeholder para evitar NameError si luego lo referenciamos
if _cp_spec is not None:
    # Solo importamos si realmente está instalado
    import importlib as _importlib
    cp = _importlib.import_module("cupy")



***No ejecutar este código a menos que se quiera comprimir***

In [ ]:
def comprimir_dividir_archivo(archivo_original, tamano_maximo=100, directorio_salida=None):
    # Asegúrate de que el archivo original existe
    archivo_original = Path(archivo_original)
    if not archivo_original.exists():
        raise FileNotFoundError(f"No se encuentra el archivo: {archivo_original}")
    
    # Si no se especifica directorio de salida, usar src/data/raw/ITP/
    if directorio_salida is None:
        # Obtener el directorio raíz del proyecto (donde está src/)
        proyecto_root = Path.cwd().parent.parent
        directorio_salida = proyecto_root /'data' / 'raw' / 'ITP' / 'ITPD_E_R03'
    else:
        directorio_salida = Path(directorio_salida)
    
    # Crear el directorio de salida si no existe
    directorio_salida.mkdir(parents=True, exist_ok=True)
    
    # Abre el archivo original en modo de lectura binaria
    with open(archivo_original, 'rb') as f_in:
        # Lee el contenido del archivo original
        contenido = f_in.read()
        
        # Determina el número de partes necesarias
        num_partes = (len(contenido) + tamano_maximo - 1) // tamano_maximo
        
        # Divide el contenido en partes y escribe cada parte comprimida
        for i in range(num_partes):
            parte = contenido[i * tamano_maximo: (i + 1) * tamano_maximo]
            archivo_salida = directorio_salida / f'ITPD_E_R03.csv.parte{i}.gz'
            with gzip.open(archivo_salida, 'wb') as f_out:
                f_out.write(parte)
            print(f"Parte {i} creada en: {archivo_salida}")

# Tamaño máximo por parte (1GB)
tamano_maximo = 810 * 1024 * 1024

try:
    # Ruta al archivo original
    archivo_original = Path(r"C:\Users\Usuario\Downloads\ITPDE_R03\ITPDE_R03.csv")
    
    # Comprimir y dividir el archivo original
    comprimir_dividir_archivo(archivo_original, tamano_maximo)
    print("Proceso completado con éxito")
except Exception as e:
    print(f"Error durante el proceso: {e}")

### ***Descomprimir, carga de datos y borrado de archivo***

La compresión se hace para poder trabajar con git sin porblemas de tamaño de ficheros.
Se descomprime, se importa y luego se borra el fichero descomprimido


# DEFINO EL AÑO

In [16]:
anio = 2000

In [17]:
"""
FASE 1: PREPARACIÓN Y CARGA DE DATOS
Este script procesa la base de datos International Trade and Production Database (ITP)
que viene dividida en múltiples archivos comprimidos, utilizando aceleración GPU
cuando está disponible.
"""

def procesar_datos_itp(year: int = anio):
    try:
        # Verificar si GPU está disponible (solo informativo en esta fase)
        gpu_disponible = torch.cuda.is_available()
        if gpu_disponible:
            print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
        else:
            print("GPU no disponible, se usará CPU")

        # Definición de rutas usando Path y la estructura de tu proyecto
        try:
            base_path = Path(__file__).parent
        except NameError:  # Estamos en un notebook
            base_path = Path.cwd().parent.parent  # Asumiendo que el notebook está en /notebooks/

        source_directory = base_path / "data" / "raw" / "ITP" / "ITPD_E_R03"
        target_directory = base_path / "data" / "processed"
        target_filename = 'ITPD_E_R03.csv'

        # Imprimir las rutas para verificación
        print(f"Directorio fuente: {source_directory}")
        print(f"Directorio destino: {target_directory}")

        # Asegurar que los directorios existen
        target_directory.mkdir(parents=True, exist_ok=True)

        # Verificar que el directorio fuente existe
        if not source_directory.exists():
            raise FileNotFoundError(f"No se encuentra el directorio fuente: {source_directory}")

        # Listar archivos comprimidos
        chunk_filenames = sorted([
            f for f in os.listdir(source_directory)
            if f.startswith('ITPD_E_R03.csv.parte') and f.endswith('.gz')
        ])

        # Control de errores: verificar que existen archivos para procesar
        if not chunk_filenames:
            raise FileNotFoundError(f"No se encontraron archivos .gz en {source_directory}")

        # Construir la ruta completa para el archivo combinado
        target_filepath = target_directory / target_filename

        # Función para descomprimir un archivo en paralelo
        def descomprimir_archivo(chunk_filename):
            chunk_filepath = source_directory / chunk_filename
            with gzip.open(chunk_filepath, 'rb') as chunk_file:
                return chunk_file.read()

        print("Combinando archivos comprimidos...")
        with open(target_filepath, 'wb') as target_file:
            # Usar ThreadPoolExecutor para paralelizar la descompresión (I/O + gzip)
            with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
                for data in tqdm(
                    executor.map(descomprimir_archivo, chunk_filenames),
                    total=len(chunk_filenames),
                    desc="Procesando archivos"
                ):
                    target_file.write(data)

                print("Leyendo archivo CSV...")

                # === columnas exactas que necesitas ===
                USECOLS = [
                    "exporter_iso3",
                    "importer_iso3",
                    "year",
                    "trade",
                    "industry_id",
                    "industry_descr",
                    "importer_name",
                    "exporter_name",
                ]
                DTYPES = {
                    "exporter_iso3": "string",
                    "importer_iso3": "string",
                    "year": "int32",
                    "trade": "float32",        # reduce memoria sin perder precisión práctica
                    "industry_id": "int32",
                    "industry_descr": "string",
                    "importer_name": "string",
                    "exporter_name": "string",
                }

                # Dask: lectura lazy del CSV combinado
                print("Usando Dask para procesamiento en paralelo (lectura filtrada)")
                dask_df = dd.read_csv(
                    target_filepath,
                    sep=",",
                    usecols=USECOLS,
                    dtype=DTYPES,
                    blocksize="128MB",
                    assume_missing=True,  # tolerante a nulos esporádicos
                )
                dask_df['year'].unique()
                print(f"Filtrando datos del año {year}...")
                dask_df = dask_df[dask_df["year"] == year]

                # Ejecutar el plan y materializar en pandas
                itp_year = dask_df.compute()

                # Tipos finales (por si Dask promovió algo)
                itp_year = itp_year.astype(DTYPES)

                # (Opcional) comprime memoria de las cadenas con 'category' si vas a agrupar mucho después
                # itp_year["exporter_iso3"]  = itp_year["exporter_iso3"].astype("category")
                # itp_year["importer_iso3"]  = itp_year["importer_iso3"].astype("category")
                # itp_year["industry_descr"] = itp_year["industry_descr"].astype("category")
                # itp_year["importer_name"]  = itp_year["importer_name"].astype("category")
                # itp_year["exporter_name"]  = itp_year["exporter_name"].astype("category")


        # Limpieza del temporal grande
        try:
            os.remove(target_filepath)
            print("Archivo temporal eliminado")
        except Exception as _e:
            print(f"Advertencia: no se pudo eliminar el temporal ({_e})")

        # 4) Países únicos importadores (GPU no aporta aquí; unique de pandas es muy rápido)
        #    Convertimos a category para memoria/velocidad y extraemos categorías ordenadas
        itp_year["importer_iso3"] = itp_year["importer_iso3"].astype("category")
        codigos_countries = list(itp_year["importer_iso3"].cat.categories)

        print(f"Total de países únicos encontrados: {len(codigos_countries)}")
        return itp_year, codigos_countries

    except Exception as e:
        print(f"Error durante el procesamiento: {e}")
        raise

if __name__ == "__main__":
    try:
        data, countries = procesar_datos_itp(year=anio)
        print("Procesamiento completado con éxito")
    except Exception as e:
        print(f"Error en la ejecución principal: {e}")


GPU disponible: NVIDIA GeForce RTX 4060 Laptop GPU
Directorio fuente: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\raw\ITP\ITPD_E_R03
Directorio destino: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed
Combinando archivos comprimidos...


Procesando archivos:  40%|████      | 4/10 [00:54<01:21, 13.52s/it]


Error durante el procesamiento: 
Error en la ejecución principal: 


In [18]:
import pandas as pd
from pathlib import Path

# Posibles alias de columnas en ITP (ajusta si conoces los exactos)
CAND_ID = ["industry_id", "industry_code", "industry", "sector_id", "sector_code", "isic", "isic4"]
CAND_NAME = ["industry_descr", "industry_name", "industry_label", "sector_name", "sector_descr"]

def _pick_col(candidates, cols):
    cols_lower = {c.lower(): c for c in cols}
    for cand in candidates:
        if cand.lower() in cols_lower:
            return cols_lower[cand.lower()]
    return None

id_col = _pick_col(CAND_ID, data.columns)
name_col = _pick_col(CAND_NAME, data.columns)

if not id_col or not name_col:
    print("⚠️ No se encontraron columnas de industria en 'data'.")
    print(f"Columnas disponibles: {list(data.columns)}")
    print(
        "Sugerencia: en la celda de carga (procesar_datos_itp), añade las columnas de industria "
        "a USECOLS/DTYPES, por ejemplo: 'industry_id' y 'industry_descr' (o sus equivalentes)."
    )
else:
    # Catálogo único, limpio y ordenado
    industrias = (
        data[[id_col, name_col]]
        .dropna()
        .drop_duplicates()
        .sort_values([id_col, name_col])
        .reset_index(drop=True)
        .rename(columns={id_col: "industry_id", name_col: "industry_descr"})
    )

    # Tipos compactos
    industrias["industry_id"] = industrias["industry_id"].astype("string")
    industrias["industry_descr"] = industrias["industry_descr"].astype("string")

    # Rutas (evitamos espacios en nombres de carpetas)
    base_path = Path.cwd().parent.parent
    target_directory = base_path / "data" / "processed" / "Dependencias_consolidadas"
    target_directory.mkdir(parents=True, exist_ok=True)

    # Salidas
    csv_path = target_directory / "industrias_id_nombre.csv"
    pq_path  = target_directory / "industrias_id_nombre.parquet"

    industrias.to_csv(csv_path, sep=";", index=False)
    industrias.to_parquet(pq_path, index=False)

    print(f"✅ Catálogo de industrias generado: {len(industrias)} filas")
    print(f"- CSV: {csv_path}")
    print(f"- Parquet: {pq_path}")




✅ Catálogo de industrias generado: 168 filas
- CSV: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\Dependencias_consolidadas\industrias_id_nombre.csv
- Parquet: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\Dependencias_consolidadas\industrias_id_nombre.parquet


# Matrices de Comercio Internacional: Creación Optimizada con Aceleración GPU

El siguiente código implementa una función optimizada para crear matrices de comercio bilateral para cada industria a partir de datos comerciales internacionales. Esta implementación utiliza técnicas avanzadas para mejorar significativamente el rendimiento:

### Características principales:

- **Aceleración por GPU**: Detecta automáticamente si hay una GPU disponible y la utiliza para acelerar el procesamiento cuando hay suficientes datos.
- **Operaciones vectorizadas**: Minimiza las iteraciones fila por fila usando enfoques vectorizados más eficientes.
- **Gestión de memoria optimizada**: Reutiliza estructuras para reducir la fragmentación y el consumo de memoria.
- **Adaptabilidad**: Ajusta automáticamente la estrategia de procesamiento según el volumen de datos y el hardware disponible.

El resultado es un diccionario donde cada clave representa una industria, y cada valor es una matriz completa de comercio bilateral entre todos los países del conjunto de datos, lo que facilita los análisis posteriores de dependencias comerciales y flujos económicos.

Este enfoque es particularmente útil para el cálculo de índices de dependencia comercial, ya que proporciona una representación eficiente de los flujos comerciales entre países para cada sector industrial.

### Limpieza de Matrices de Comercio: Eliminación de Flujos No Significativos

El siguiente código implementa una función para eliminar del análisis aquellos países con flujos comerciales no significativos. Este paso es crucial para mejorar la precisión del análisis de dependencias comerciales, ya que permite enfocarse en relaciones económicas realmente relevantes.

**Funcionamiento**:

- **Umbral de significancia**: Establece un umbral mínimo (por defecto 0.05%) del comercio mundial total. Cualquier flujo comercial por debajo de este umbral se considera no significativo y se convierte a cero.
- **Identificación de países no relevantes**: Detecta países que, después de aplicar el umbral, no tienen flujos comerciales significativos (tanto como exportadores como importadores).
- **Limpieza de la matriz**: Elimina estos países de la matriz, reduciendo su dimensionalidad y concentrando el análisis en actores relevantes del comercio internacional.

Esta función de preprocesamiento es especialmente importante para estudios de dependencia económica, ya que:
1. Reduce el ruido en los datos al eliminar relaciones comerciales marginales
2. Mejora la eficiencia computacional al trabajar con matrices más pequeñas
3. Permite concentrarse en dependencias significativas que podrían representar vulnerabilidades reales

El resultado es una matriz "limpia" que contiene únicamente los países y flujos comerciales relevantes para el análisis posterior.

In [9]:
from pathlib import Path
import pandas as pd
import numpy as np
from typing import List, Dict

def crear_matriz_comercio_optimizado(
    grouped_data, 
    codigos_paises: List[str],
    target_directory: Path | None = None
) -> Dict[str, pd.DataFrame]:
    """
    Crea matrices de comercio bilateral (exportador x importador) para cada industria,
    usando operaciones vectorizadas (pivot) y deja un CSV de totales por país/industria.

    Args:
        grouped_data: resultado de data.groupby('industry_descr') (o similar).
        codigos_paises: lista de ISO3 a usar como índice/columnas de referencia.
        target_directory: carpeta donde guardar 'totales_comercio_por_pais_industria.csv'.

    Returns:
        dict: {industry -> DataFrame (index=exporter_iso3, columns=importer_iso3)}
    """
    # Resolver destino si no viene dado
    if target_directory is None:
        try:
            base_path = Path(__file__).parent
        except NameError:  # notebook
            base_path = Path.cwd().parent.parent
        target_directory = base_path / "data" / "processed" / "Dependencias_consolidadas"
    target_directory.mkdir(parents=True, exist_ok=True)

    matrices: Dict[str, pd.DataFrame] = {}
    totales_registros = []

    # Columnas requeridas con alias tolerantes
    cols = grouped_data.obj.columns
    trade_col = None
    for cand in ("trade", "value", "trade_value"):
        if cand in cols:
            trade_col = cand
            break
    required = {"exporter_iso3", "importer_iso3"}
    if trade_col is None or not required.issubset(cols):
        raise ValueError(
            f"Faltan columnas. Necesito {required} y una de ['trade','value','trade_value'].\n"
            f"Columnas disponibles: {list(cols)}"
        )

    # Preconstruir un DataFrame vacío (para industrias sin datos válidos)
    template = pd.DataFrame(
        0.0, index=codigos_paises, columns=codigos_paises, dtype=np.float32
    )

    # Bucle por industria (grouped_data es p.groupby(...))
    for industry, group in grouped_data:
        # Filtrar a países en la lista (evita claves no esperadas)
        g = group[
            group["exporter_iso3"].isin(codigos_paises) &
            group["importer_iso3"].isin(codigos_paises)
        ]

        if g.empty:
            matrices[industry] = template.copy()
            # totales = 0 para todos los países en esta industria
            totales_registros.extend(
                {"pais": p, "industria": industry, "valor_importado": 0.0}
                for p in codigos_paises
            )
            continue

        # Pivot vectorizado y suma de duplicados
        mat = (
            g.pivot_table(
                index="exporter_iso3",
                columns="importer_iso3",
                values=trade_col,
                aggfunc="sum",
                fill_value=0.0,
                observed=False
            )
            .reindex(index=codigos_paises, columns=codigos_paises, fill_value=0.0)
            .astype(np.float32)
        )

        matrices[industry] = mat

        # Totales por país importador (columna): suma vectorizada
        totales = mat.sum(axis=0)  # importaciones totales por país en esta industria
        totales_registros.extend(
            {"pais": pais, "industria": industry, "valor_importado": float(totales[pais])}
            for pais in codigos_paises
        )

    # Guardar totales
    df_totales = pd.DataFrame(totales_registros)
    df_totales.to_csv(
        target_directory / "totales_comercio_por_pais_industria.csv",
        index=False, sep=";"
    )
    print(f"Archivo CSV de totales guardado en: {target_directory/'totales_comercio_por_pais_industria.csv'}")

    return matrices


# Define la lista de códigos de países
codigos_paises = sorted(data['importer_iso3'].unique().tolist())

# Llama a la función optimizada (mismo API esperado)
matrices_comercio = crear_matriz_comercio_optimizado(
    data.groupby('industry_descr'),
    codigos_paises=codigos_paises,
    # target_directory=Path("...")  # opcional
)

def eliminar_filas_columnas_cero(df: pd.DataFrame, threshold_pct: float = 0.005) -> pd.DataFrame:
    """Filtra una matriz aplicando umbral relativo por país importador (columna)."""
    # Umbral por columna: total_col * pct
    col_totals = df.sum(axis=0)
    thresholds = col_totals * float(threshold_pct)

    # Aplicar máscara vectorizada (alineación por columnas)
    df_filtered = df.where(df >= thresholds, 0.0)

    # Países con filas y columnas a cero tras umbral
    zero_rows = df_filtered.index[df_filtered.sum(axis=1) == 0.0]
    zero_cols = df_filtered.columns[df_filtered.sum(axis=0) == 0.0]
    to_drop = list(set(zero_rows) & set(zero_cols))

    return df_filtered.drop(index=to_drop, columns=to_drop)


Archivo CSV de totales guardado en: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\Dependencias_consolidadas\totales_comercio_por_pais_industria.csv


# Cálculo de dependencias

## 🚀 Qué hacen estas funciones (y por qué son más rápidas)

Este bloque sustituye la enumeración exhaustiva de combinaciones de intermediarios por una **búsqueda en profundidad (DFS) con poda** sobre un **grafo ralo**. Mantiene la **misma firma y estructura de salida** que tu `process_country_pair` original, pero calcula **caminos simples** (sin repetir nodos) de forma mucho más eficiente.

### Ideas clave
- **Matriz de transición `T`**: se normaliza la matriz de comercio `X` por columnas:  
  `T[u, v] = X[u, v] / sum_a X[a, v]`. Esto es exactamente el cociente que ya usabas en `calculate_path_dependency`.
- **Grafo ralo**: solo se conservan aristas con peso `T[u,v] ≥ eps_edge` y, opcionalmente, los **top_m** destinos más fuertes por nodo. Así reducimos el branching.
- **DFS con poda por cota superior**: en cada expansión, si la **mejor contribución posible restante** es inferior a `eps_contrib`, se corta la rama. Evita explorar caminos que no aportarían nada relevante.

---

### Funciones

#### `_prepare_graph_local(X_clean, eps_edge, top_m)`
- **Entrada:** matriz de comercio `X_clean` (cuadrada), umbral de arista `eps_edge` y límite de vecinos `top_m`.
- **Salida:**
  - `T`: matriz de transición normalizada (float32).
  - `neighbors`: lista (por nodo) de vecinos salientes **ordenados por peso descendente**, filtrados por `eps_edge` y truncados a `top_m`.
  - `max_out`: para cada nodo, el **peso máximo** de sus aristas salientes (sirve para la cota en la poda).
- **Rol:** preparar una representación esbelta del grafo para acelerar el DFS.

#### `_sum_simple_paths_DFS(src, dst, T, neighbors, max_out, Lmax, eps_contrib)`
- **Calcula la suma de contribuciones** de **todos los caminos simples** de `src` a `dst` con longitudes `2..Lmax`.
- **Poda por cota:** si `producto_actual * (max_out[u] ** pasos_restantes) < eps_contrib`, no se expande esa rama.
- **Devuelve:**
  - `indirect_total`: suma de todas las contribuciones indirectas (longitud ≥ 2).
  - `deps_by_len`: diccionario `{longitud -> contribución}`, con `deps_by_len[1] = T[src, dst]` para coherencia con tu salida.
  - `significant_paths`: lista (opcional) con rutas fuertes si se activa su recopilación (por defecto vacía para rendimiento).

#### `process_country_pair(i, j, X_clean, denominators, country_names, max_possible_length, convergence_threshold, path_strength_threshold)`
- **Entrada:** índices `i` (importador), `j` (exportador), matriz `X_clean`, denominadores por columna, nombres de países y parámetros.
- **Pasos:**
  1. **Dependencia directa (DD):** `X[j, i] / denominators[i]` (igual que antes).
  2. **Grafo ralo:** crea `T`, `neighbors`, `max_out` con `_prepare_graph_local` usando  
     `eps_edge = path_strength_threshold` (umbral de arista).
  3. **Dependencia indirecta (DI):** suma **caminos simples** `j → … → i` con `_sum_simple_paths_DFS` hasta `max_possible_length`, podando por `eps_contrib = path_strength_threshold`.
  4. **Convergencia por longitud:** si al añadir la contribución de la siguiente longitud el aumento es `< convergence_threshold`, se considera convergido antes de `Lmax`.
- **Salida (idéntica en forma a la original):**
  - `result`: dict con `importador`, `exportador`, `trade_value`, `dependencia_directa`, `dependencia_indirecta`, `dependencia_total`, `longitud_optima`, `dependencias_por_longitud`.
  - `pair_key`: `"exportador->importador"`.
  - `top_dependency`: tu tupla `(importador, exportador, DD, DI, DT, longitud_optima)`.
  - `significant_paths`: lista (vacía por defecto para velocidad).
  - `length_converged`: longitud donde se detuvo por convergencia (0 si no aplica).

---

### Parámetros importantes (y cómo afinarlos)
- `max_possible_length` (`Lmax`): longitud máxima de los caminos simples. Típico: **3–5**.
- `path_strength_threshold`:
  - Como **`eps_edge`**: filtra aristas débiles al construir el grafo.
  - Como **`eps_contrib`**: poda ramas cuya contribución restante es irrelevante.
- `top_m` (en `_prepare_graph_local`): **máximo de vecinos salientes** por nodo que se exploran (ordenados por peso).  
  - 10–20 ⇒ muy rápido; 30–50 ⇒ más cobertura; `None` ⇒ sin límite (más lento).
- `convergence_threshold`: criterio para parar por longitud si los incrementos son despreciables.

---

### Exactitud vs. rendimiento
- Se siguen sumando **caminos simples** (como querías), pero:
  - Descartamos aristas y ramas **que no moverían el resultado** más allá de tus umbrales.
  - El orden de expansión por aristas fuertes permite **podar antes**.
- Ajusta `path_strength_threshold` y `top_m` para el mejor equilibrio **tiempo ↔ precisión**.  
  Empieza con `path_strength_threshold = 1e-3` (o tu valor actual) y `top_m = 20`.

> Si más adelante necesitas listar **rutas** para un par concreto en el dashboard, podemos activar la captura de `significant_paths` solo **on-demand**, sin penalizar el cálculo batch.



In [19]:


def calculate_path_dependency(X_clean, path, denominators):
    """Calcula la dependencia de un camino específico."""
    fuerza_camino = 1.0
    for k in range(len(path) - 1):
        if denominators[path[k+1]] > 0:
            fuerza_camino *= X_clean[path[k], path[k+1]] / denominators[path[k+1]]
        else:
            fuerza_camino = 0  # Si el denominador es cero, la fuerza del camino es cero
    return fuerza_camino

def calculate_intermediary_centrality(intermediary_frequency, intermediary_strength, country_names):
    """
    Calcula métricas de centralidad para intermediarios.
    
    Esta función es la misma que la original.
    """
    # Implementación existente
    centrality = []
    
    # Normalizar
    max_freq = max(intermediary_frequency.values()) if intermediary_frequency.values() else 1
    max_strength = max(intermediary_strength.values()) if intermediary_strength.values() else 1
    
    for country in country_names:
        norm_freq = intermediary_frequency[country] / max_freq if max_freq > 0 else 0
        norm_strength = intermediary_strength[country] / max_strength if max_strength > 0 else 0
        
        combined_score = 0.4 * norm_freq + 0.6 * norm_strength
        
        centrality.append((country, intermediary_frequency[country], 
                          intermediary_strength[country], combined_score))
    
    centrality.sort(key=lambda x: x[3], reverse=True)
    return centrality


from itertools import combinations
import numpy as np

def process_country_pair(i, j, X_clean, denominators, country_names,
                         max_possible_length, convergence_threshold,
                         path_strength_threshold, T):
    """
    Exactamente la misma definición que antes (combinations), pero:
    - Usa T (matriz de transición) precalculada.
    - Vectoriza la longitud 2.
    - Mantiene mismos retornos y campos.
    """
    n = X_clean.shape[0]

    # Dependencia directa (igual que antes)
    trade_value = X_clean[j, i]
    direct_dependency = (X_clean[j, i] / denominators[i]) if denominators[i] > 0 else 0.0

    dependencies_by_length = {1: direct_dependency}
    significant_paths = []
    current_total = direct_dependency
    indirect_total = 0.0
    length = 1

    # Candidatos de intermediarios (igual que antes)
    middle = [k for k in range(n) if k != i and k != j]

    # ---- L = 2 (vectorizado, EXACTO) ----
    if max_possible_length >= 2 and middle:
        row_j = T[j, :]
        col_i = T[:, i]
        mask = np.ones(n, dtype=bool)
        mask[[i, j]] = False

        # suma exacta: sum_k T[j,k]*T[k,i], k!=i,j
        di2 = np.dot(row_j[mask], col_i[mask])
        dependencies_by_length[2] = float(di2)
        indirect_total += float(di2)
        current_total = direct_dependency + indirect_total
        length = 2

        # caminos significativos para L=2 (mismo umbral)
        if path_strength_threshold > 0:
            ks = np.nonzero(row_j * col_i > path_strength_threshold)[0]
            ks = [k for k in ks if k != i and k != j]
            if ks:
                for k in ks:
                    significant_paths.append({
                        'exportador': country_names[j],
                        'importador': country_names[i],
                        'intermediarios': [country_names[k]],
                        'fuerza': float(row_j[k] * col_i[k]),
                        'longitud': 2
                    })

        # criterio de convergencia
        if abs(current_total - direct_dependency) < convergence_threshold or max_possible_length == 2:
            # ordenar y devolver
            significant_paths.sort(key=lambda x: x['fuerza'], reverse=True)
            pair_key = f"{country_names[j]}->{country_names[i]}"
            result = {
                'importador': country_names[i],
                'exportador': country_names[j],
                'trade_value': trade_value,
                'dependencia_directa': direct_dependency,
                'dependencia_indirecta': indirect_total,
                'dependencia_total': direct_dependency + indirect_total,
                'longitud_optima': 2,
                'dependencias_por_longitud': dependencies_by_length
            }
            return {
                'pair_key': pair_key,
                'result': result,
                'top_dependency': (country_names[i], country_names[j],
                                   direct_dependency, indirect_total,
                                   direct_dependency + indirect_total, 2),
                'significant_paths': significant_paths,
                'length_converged': 2
            }

    # ---- L >= 3 (exacto con combinations, pero usando T) ----
    for L in range(3, max_possible_length + 1):
        DI_ij_L = 0.0
        for interms in combinations(middle, L - 1):
            path = (j,) + interms + (i,)
            # producto exacto de T a lo largo del camino
            prod = 1.0
            for a, b in zip(path[:-1], path[1:]):
                w = T[a, b]
                if w == 0.0:
                    prod = 0.0
                    break
                prod *= w

            DI_ij_L += prod

            if prod > path_strength_threshold:
                significant_paths.append({
                    'exportador': country_names[j],
                    'importador': country_names[i],
                    'intermediarios': [country_names[x] for x in interms],
                    'fuerza': float(prod),
                    'longitud': L
                })

        dependencies_by_length[L] = float(DI_ij_L)
        indirect_total += float(DI_ij_L)

        prev_total = current_total
        current_total = direct_dependency + indirect_total
        length = L

        if L > 1 and abs(current_total - prev_total) < convergence_threshold:
            break

    # Final
    significant_paths.sort(key=lambda x: x['fuerza'], reverse=True)
    total_dependency = direct_dependency + indirect_total
    pair_key = f"{country_names[j]}->{country_names[i]}"
    result = {
        'importador': country_names[i],
        'exportador': country_names[j],
        'trade_value': trade_value,
        'dependencia_directa': direct_dependency,
        'dependencia_indirecta': indirect_total,
        'dependencia_total': total_dependency,
        'longitud_optima': length,
        'dependencias_por_longitud': dependencies_by_length
    }
    return {
        'pair_key': pair_key,
        'result': result,
        'top_dependency': (country_names[i], country_names[j],
                           direct_dependency, indirect_total, total_dependency, length),
        'significant_paths': significant_paths,
        'length_converged': length if length > 1 else 0
    }




def calculate_all_dependencies_parallel(X, country_names=None, convergence_threshold=0.01, 
                                       max_possible_length=3, 
                                       path_strength_threshold=0.001, n_jobs=None, use_gpu=True, 
                                       debug_mode=False):
    """
    Versión paralelizada del cálculo de dependencias que mantiene EXACTAMENTE
    la misma salida que la versión original.
    
    El parámetro debug_mode permite verificar que el número de dependencias
    coincida con la versión original.
    """
    """
    Versión paralelizada del cálculo de dependencias.
    
    Parameters adicionales:
    -----------------------
    n_jobs : int, opcional
        Número de trabajos paralelos. Si es None, usa todos los núcleos disponibles.
    use_gpu : bool, default=True
        Si se debe intentar usar GPU para acelerar algunos cálculos.
    """
    n = X.shape[0]

    if country_names is None:
        country_names = [f"País {i}" for i in range(n)]

    if len(country_names) != n:
        raise ValueError(f"La longitud de country_names ({len(country_names)}) no coincide con la dimensión de X ({n})")

    # Configurar paralelización
    if n_jobs is None:
        n_jobs = multiprocessing.cpu_count()
    
    # Verificar disponibilidad de GPU
    gpu_available = torch.cuda.is_available() and use_gpu
    X_clean = X

    denom = X_clean.sum(axis=0, dtype=np.float64)
    denom[denom == 0.0] = np.inf
    T = (X_clean / denom).astype(np.float64, copy=False)

    denominators = np.sum(X, axis=0)

    # Acelerar cálculos directos con GPU si está disponible
    if gpu_available:
        # Transferir datos a GPU
        X_gpu = torch.tensor(X_clean, device='cuda', dtype=torch.float32)
        denom_gpu = torch.tensor(denominators, device='cuda', dtype=torch.float32)
        
        # Calcular dependencias directas en forma vectorizada
        direct_deps = torch.zeros_like(X_gpu)
        for i in range(n):
            # Evitar división por cero
            if denom_gpu[i] > 0:
                direct_deps[:, i] = X_gpu[:, i] / denom_gpu[i]
        
        # Transferir resultados de vuelta a CPU
        direct_dependencies = direct_deps.cpu().numpy()
        
        # Usar estas dependencias directas precalculadas en el procesamiento posterior
        # (Aunque en esta implementación seguimos calculándolas en process_country_pair para
        # mantener cambios mínimos en el código)

    # Estructura de resultados extendida
    results = {
        'dependencies': [],
        'top_dependencies': [],
        'avg_dependencies': {},
        'length_distribution': np.zeros(max_possible_length),
        'critical_intermediaries': {},     # Intermediarios críticos por relación
        'intermediary_frequency': {},      # Frecuencia de países como intermediarios
        'critical_paths': [],              # Rutas críticas completas
        'intermediary_strength': {}        # Fuerza de cada país como intermediario
    }
    
    # Inicializar contadores para intermediarios
    for country in country_names:
        results['intermediary_frequency'][country] = 0
        results['intermediary_strength'][country] = 0.0

    # Preparar pares de países para procesamiento paralelo 
    # Mantenemos la misma estructura de iteración del código original
    # Primero por importador (i) y luego por exportador (j)
        # Preparar pares de países (sin tqdm)
    country_pairs = [(i, j) for i in range(n) for j in range(n) if i != j]

    # Procesar pares de países en paralelo (sin barra de progreso)
    with Parallel(n_jobs=n_jobs) as parallel:
        pair_results = parallel(
            delayed(process_country_pair)(
                i, j, X_clean, denom, country_names, 
                max_possible_length, convergence_threshold, path_strength_threshold,
                T  # <-- NUEVO ARGUMENTO
            )
            for i, j in country_pairs
        )


        
    # Agrupar resultados por país importador
    results_by_importer = {}
    for res in pair_results:
        importer = res['result']['importador']
        if importer not in results_by_importer:
            results_by_importer[importer] = []
        results_by_importer[importer].append(res)
    
    # Recolectar critical paths de todos los pares para ordenarlos después
    all_critical_paths = []
    
    # Procesar los resultados manteniendo el mismo orden que el código original
    for i in range(n):
        importer = country_names[i]
        total_dep = 0.0
        num_deps = 0
        
        if importer in results_by_importer:
            for res in results_by_importer[importer]:
                # Agregar a dependencies
                results['dependencies'].append(res['result'])
                
                # Agregar a top_dependencies
                results['top_dependencies'].append(res['top_dependency'])
                
                # Actualizar critical_intermediaries
                results['critical_intermediaries'][res['pair_key']] = res['significant_paths']
                
                # Recolectar critical paths
                all_critical_paths.extend(res['significant_paths'])
                
                # Actualizar length_distribution si convergió
                if res['length_converged'] > 1:
                    results['length_distribution'][res['length_converged'] - 1] += 1
                
                # Actualizar dependencia promedio
                total_dep += res['result']['dependencia_total']
                num_deps += 1
                
                # Actualizar estadísticas de intermediarios
                for path in res['significant_paths']:
                    for idx, interm in enumerate(path['intermediarios']):
                        # Incrementar frecuencia
                        results['intermediary_frequency'][interm] += 1
                        
                        # Incrementar fuerza ponderada
                        weight_factor = 1.0 / (idx + 1)
                        results['intermediary_strength'][interm] += path['fuerza'] * weight_factor
        
        # Guardar dependencia promedio para este importador
        results['avg_dependencies'][importer] = total_dep / num_deps if num_deps > 0 else 0
    
    # Añadir y ordenar los critical paths (igual que el original)
    results['critical_paths'] = all_critical_paths
    
    # Ordenar top dependencies
    results['top_dependencies'].sort(key=lambda x: x[4], reverse=True)
    results['top_dependencies'] = results['top_dependencies'][:90]
    
    # Ordenar critical_paths por fuerza
    results['critical_paths'].sort(key=lambda x: x['fuerza'], reverse=True)
    
    # Calcular métricas de centralidad para intermediarios
    results['intermediary_centrality'] = calculate_intermediary_centrality(
        results['intermediary_frequency'], 
        results['intermediary_strength'],
        country_names
    )
    
    return results

# Para mantener compatibilidad, redefinimos la función original
# para que utilice la versión paralelizada
def calculate_all_dependencies(X, country_names=None, convergence_threshold=0.01, 
                              max_possible_length=3, threshold_pct=0.01, 
                              path_strength_threshold=0.001):
    """
    Calcula todas las dependencias entre países con análisis de intermediarios críticos.
    
    Esta función mantiene EXACTAMENTE la misma firma y resultados que la original,
    pero utiliza internamente paralelización y GPU para acelerar los cálculos.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Matriz de comercio
    country_names : list, opcional
        Nombres de los países
    convergence_threshold : float, default=0.01
        Umbral para determinar la convergencia
    max_possible_length : int, default=5
        Longitud máxima de caminos a considerar
    threshold_pct : float, default=0.01
        Umbral para filtrar valores de comercio insignificantes (porcentaje)
    path_strength_threshold : float, default=0.001
        Umbral mínimo para considerar una ruta como significativa
        
    Returns:
    --------
    dict
        Diccionario con todos los resultados del análisis
    """
    # Determinar si usar paralelización basado en el tamaño del problema
    use_parallel = X.shape[0] > 5  # Para matrices muy pequeñas no vale la pena paralelizar
    
    # Verificar disponibilidad de GPU
    use_gpu = torch.cuda.is_available()
    
    # Aquí agregas el segundo log
    #print(f"Usando paralelización: {use_parallel}, GPU disponible: {use_gpu}")
    #if use_gpu:
    #    print(f"GPU en uso: {torch.cuda.get_device_name(0)}")
    
    # Configurar número de trabajos para paralelización
    n_countries = X.shape[0]
    n_jobs = min(multiprocessing.cpu_count(), n_countries)  # Limitar al número de países
    
    
    if use_parallel:
        # Usar la versión paralelizada
        return calculate_all_dependencies_parallel(
            X, country_names, convergence_threshold, 
            max_possible_length, path_strength_threshold,
            n_jobs=n_jobs, use_gpu=use_gpu, debug_mode=False
        )
    else:
        # Para matrices muy pequeñas, usar un solo proceso
        return calculate_all_dependencies_parallel(
            X, country_names, convergence_threshold, 
            max_possible_length, path_strength_threshold,
            n_jobs=1, use_gpu=use_gpu, debug_mode=False
        )


## Generar y guardar resultados de dependencias

In [20]:
# Diccionario para guardar todos los resultados
all_results = {}

total_industrias = len(matrices_comercio)
completadas = 0
print(f"Completadas: {completadas}/{total_industrias}", end="\r", flush=True)

# Procesar cada industria
for industry, mat in matrices_comercio.items():
    # Limpiar matriz
    mat_clean = eliminar_filas_columnas_cero(mat)
    
    # Si la matriz limpia está vacía o es muy pequeña, continuamos con la siguiente
    if mat_clean.shape[0] < 2:
        # no cuenta como completada (no se calculan dependencias)
        continue
        
    # Convertir a numpy array y obtener nombres de países
    X = mat_clean.values
    country_names = mat_clean.columns.tolist()

    # Calcular dependencias
    results = calculate_all_dependencies(X, country_names)
    
    # Guardar resultados
    all_results[industry] = {
        'results': results,
        'country_names': country_names,
        'matrix_shape': mat_clean.shape
    }

    # Actualizar contador y mostrarlo en la misma línea
    completadas += 1
    print(f"Completadas: {completadas}/{total_industrias}", end="\r", flush=True)

# Imprimir línea final (nuevo salto) al terminar
print(f"\nCompletadas: {completadas}/{total_industrias} ✅")


KeyboardInterrupt: 

## Generar y guadar datos de intermediarios Críticos

In [12]:
def get_top_intermediaries(results, top_n=10):
    """
    Obtiene los países más importantes como intermediarios.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis con intermediarios
    top_n : int
        Número de países a mostrar
        
    Returns:
    --------
    list
        Lista de tuplas (país, score, frecuencia, fuerza) ordenadas por importancia
    """
    # La estructura de intermediary_centrality ha cambiado en la nueva implementación
    # Ahora es una lista de tuplas (país, frecuencia, fuerza, score_combinado)
    if isinstance(results['intermediary_centrality'], list):
        # Nueva implementación: ya está ordenada por score combinado
        return results['intermediary_centrality'][:top_n]
    else:
        # Implementación anterior (por compatibilidad)
        centrality_scores = [(country, metrics) 
                            for country, metrics in results['intermediary_centrality'].items()]
        centrality_scores.sort(key=lambda x: x[1]['centrality_score'], reverse=True)
        return centrality_scores[:top_n]

def analyze_country_intermediary_role(results, country):
    """
    Analiza el papel de un país específico como intermediario.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis con intermediarios
    country : str
        País a analizar
        
    Returns:
    --------
    dict
        Información detallada sobre el rol del país como intermediario
    """
    # Verificar si el país existe en los resultados
    if isinstance(results['intermediary_centrality'], list):
        # Nueva implementación - lista de tuplas
        country_entry = next((entry for entry in results['intermediary_centrality'] 
                             if entry[0] == country), None)
        
        if country_entry is None:
            return {"error": f"El país {country} no está en los resultados"}
            
        # Extraer métricas
        centrality = {
            "frequency": country_entry[1],
            "strength": country_entry[2],
            "centrality_score": country_entry[3]
        }
    else:
        # Implementación anterior
        if country not in results['intermediary_centrality']:
            return {"error": f"El país {country} no está en los resultados"}
        centrality = results['intermediary_centrality'][country]
    
    # Encontrar las rutas más importantes donde este país actúa como intermediario
    top_paths = []
    for path in results['critical_paths']:
        if country in path['intermediarios']:
            top_paths.append(path)
    
    # Ordenar por fuerza del camino
    top_paths.sort(key=lambda x: x['fuerza'], reverse=True)
    
    # Filtrar los 10 caminos más importantes
    top_paths = top_paths[:10]
    
    return {
        "metrics": centrality,
        "top_paths": top_paths,
        "role_summary": {
            "total_paths": len(top_paths),
            "average_path_strength": sum(p['fuerza'] for p in top_paths) / len(top_paths) if top_paths else 0,
            "max_path_strength": max(p['fuerza'] for p in top_paths) if top_paths else 0,
            "unique_exporters": len(set(p['exportador'] for p in top_paths)),
            "unique_importers": len(set(p['importador'] for p in top_paths))
        }
    }

def summarize_country_dependencies(results, country, top_n=5):
    """
    Genera un resumen de las dependencias comerciales de un país específico.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis de dependencias
    country : str
        País a analizar
    top_n : int
        Número de dependencias principales a mostrar
        
    Returns:
    --------
    dict
        Resumen de dependencias del país
    """
    # Dependencias como importador (otros países exportan a este país)
    import_dependencies = []
    for dep in results['dependencies']:
        if dep['importador'] == country:
            import_dependencies.append(dep)
    
    # Ordenar por dependencia total
    import_dependencies.sort(key=lambda x: x['dependencia_total'], reverse=True)
    
    # Dependencias como exportador (este país exporta a otros)
    export_dependencies = []
    for dep in results['dependencies']:
        if dep['exportador'] == country:
            export_dependencies.append(dep)
    
    # Ordenar por dependencia total
    export_dependencies.sort(key=lambda x: x['dependencia_total'], reverse=True)
    
    # Calcular dependencia promedio
    avg_dependency = results['avg_dependencies'].get(country, 0)
    
    # Analizar el papel como intermediario
    intermediary_role = None
    if isinstance(results['intermediary_centrality'], list):
        # Nueva implementación
        intermediary_info = next((x for x in results['intermediary_centrality'] if x[0] == country), None)
        if intermediary_info:
            intermediary_role = {
                "frequency": intermediary_info[1],
                "strength": intermediary_info[2],
                "centrality_score": intermediary_info[3],
                "rank": next((i+1 for i, x in enumerate(results['intermediary_centrality']) 
                             if x[0] == country), None)
            }
    else:
        # Implementación anterior
        if country in results['intermediary_centrality']:
            intermediary_role = results['intermediary_centrality'][country]
            # Calcular rango
            countries_sorted = sorted(results['intermediary_centrality'].keys(), 
                                     key=lambda x: results['intermediary_centrality'][x]['centrality_score'],
                                     reverse=True)
            intermediary_role["rank"] = countries_sorted.index(country) + 1
    
    return {
        "country": country,
        "avg_dependency": avg_dependency,
        "top_import_dependencies": import_dependencies[:top_n],
        "top_export_dependencies": export_dependencies[:top_n],
        "total_import_dependencies": len(import_dependencies),
        "total_export_dependencies": len(export_dependencies),
        "intermediary_role": intermediary_role
    }

def identify_critical_trade_relationships(results, threshold=0.7, min_paths=3):
    """
    Identifica relaciones comerciales críticas con alta dependencia.
    
    Parameters:
    -----------
    results : dict
        Resultados del análisis de dependencias
    threshold : float
        Umbral de dependencia para considerar una relación como crítica
    min_paths : int
        Número mínimo de caminos alternativos para evitar criticidad
        
    Returns:
    --------
    list
        Lista de relaciones críticas
    """
    critical_relationships = []
    
    # Analizar cada dependencia
    for dep in results['dependencies']:
        if dep['dependencia_total'] >= threshold:
            # Buscar caminos alternativos
            pair_key = f"{dep['exportador']}->{dep['importador']}"
            alternative_paths = []
            
            if pair_key in results['critical_intermediaries']:
                alternative_paths = results['critical_intermediaries'][pair_key]
            
            # Si hay pocos caminos alternativos, es una relación crítica
            if len(alternative_paths) < min_paths:
                critical_relationships.append({
                    "exportador": dep['exportador'],
                    "importador": dep['importador'],
                    "dependencia_total": dep['dependencia_total'],
                    "dependencia_directa": dep['dependencia_directa'],
                    "caminos_alternativos": len(alternative_paths),
                    "criticidad": 1.0 - (len(alternative_paths) / min_paths) 
                                     if min_paths > 0 else 1.0
                })
    
    # Ordenar por criticidad
    critical_relationships.sort(key=lambda x: x['criticidad'], reverse=True)
    
    return critical_relationships

In [13]:
interm = get_top_intermediaries(all_results['Aircraft and spacecraft']['results'])
role = analyze_country_intermediary_role(results, 'DEU')

## Creo Dataframes para guardar resultados

In [14]:
import pandas as pd

def create_dependencies_dataframe(all_results):
    """
    Crea un DataFrame con los resultados de dependencias para todas las industrias.
    
    Parameters:
    -----------
    all_results : dict
        Diccionario con los resultados por industria
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame con las columnas: industria, importador, exportador, 
        dependencia_total, dependencia_directa, dependencia_indirecta, longitud_optima
    """
    # Lista para almacenar los datos de todas las industrias
    all_data = []
    
    # Procesar cada industria
    for industry, data in all_results.items():
        # Obtener los resultados de esta industria
        industry_results = data['results']['dependencies']
        
        # Añadir cada fila de resultados
        for result in industry_results:
            row = {
                'industria': industry,
                'importador': result['importador'],
                'exportador': result['exportador'],
                'dependencia_total': result['dependencia_total'],
                'dependencia_directa': result['dependencia_directa'],
                'dependencia_indirecta': result['dependencia_indirecta'],
                'trade_value': result['trade_value'],
                'longitud_optima': result['longitud_optima']
            }
            all_data.append(row)
    
    # Crear el DataFrame
    df = pd.DataFrame(all_data)
    
    # Ordenar el DataFrame
    df = df.sort_values(['industria', 'dependencia_total'], ascending=[True, False])
    
    return df

# Ejemplo de uso:
df = create_dependencies_dataframe(all_results)

# Definir el nuevo mapeo de nombres de columnas
nuevo_nombres = {
    'industria': 'industry',
    'importador': 'dependent_country',
    'exportador': 'supplier_country',
    'dependencia_total': 'dependency_value',
    'dependencia_directa': 'direct_dependency',
    'dependencia_indirecta': 'indirect_dependency',
    'trade_value': 'trade_value',
    'longitud_optima': 'longitud_optima'
}

# Renombrar las columnas
df = df.rename(columns=nuevo_nombres)

# Rutas (evitamos espacios en nombres de carpetas)
base_path = Path.cwd().parent.parent
target_directory = base_path / "data" / "processed" / "dependencias_consolidadas"

ruta_archivo = target_directory / f"dependencias{anio}.csv.gz"


# Guardar como gzip
with gzip.open(ruta_archivo, 'wt', encoding='utf-8') as f:
    df.to_csv(f, sep=";", index=False)

print(f"DataFrame guardado correctamente en: {ruta_archivo}")

DataFrame guardado correctamente en: c:\Users\Usuario\Documents\Github\Seguridad Economica\data\processed\dependencias_consolidadas\dependencias2001.csv.gz


# Lo que propone CHATGPT para hacer la seccion 6 del paper

In [ ]:
import os
import pandas as pd
import gzip
from pathlib import Path


current_dir = Path.cwd().parent.parent
output_dir = current_dir / "data" / "processed" / "ficheros_paper"
os.makedirs(output_dir, exist_ok=True)

# -----------------------------------------------
# 1. Dependencias completas (industria-país-par)
df = create_dependencies_dataframe(all_results)

# Renombrado (ya implementado antes)
df = df.rename(columns={
    'industria': 'industry',
    'importador': 'dependent_country',
    'exportador': 'supplier_country',
    'dependencia_total': 'dependency_value',
    'dependencia_directa': 'direct_dependency',
    'dependencia_indirecta': 'indirect_dependency',
    'trade_value': 'trade_value',
    'longitud_optima': 'optimal_length'
})

# Guardar
df.to_csv(os.path.join(output_dir, "dependencies_full.csv.gz"), sep=";", index=False, compression="gzip")

# -----------------------------------------------
# 2. Dependencias ponderadas bilateralmente
def safe_weighted_average(group):
    if group['trade_value'].sum() == 0:
        return np.nan
    return np.average(group['dependency_value'], weights=group['trade_value'])

weighted_dependencies = df.groupby(['dependent_country', 'supplier_country']).apply(
    safe_weighted_average
).reset_index(name='weighted_dependency')

weighted_dependencies.dropna().to_csv(
    os.path.join(output_dir, "weighted_dependencies.csv.gz"), sep=";", index=False, compression="gzip"
)

# -----------------------------------------------
# 3. Relaciones críticas
critical_relationships = []

for industry, data in all_results.items():
    crits = identify_critical_trade_relationships(data['results'], threshold=0.7, min_paths=3)
    for c in crits:
        c['industry'] = industry
    critical_relationships.extend(crits)

df_critical = pd.DataFrame(critical_relationships)
df_critical.to_csv(os.path.join(output_dir, "critical_relations.csv.gz"), sep=";", index=False, compression="gzip")

# -----------------------------------------------
# 4. Intermediarios por industria
intermediary_centrality_all = []

for industry, data in all_results.items():
    for c in data['results']['intermediary_centrality']:
        intermediary_centrality_all.append({
            'industry': industry,
            'country': c[0],
            'frequency': c[1],
            'strength': c[2],
            'centrality_score': c[3]
        })

df_centrality = pd.DataFrame(intermediary_centrality_all)
df_centrality.to_csv(os.path.join(output_dir, "intermediary_roles.csv.gz"), sep=";", index=False, compression="gzip")

# -----------------------------------------------
# 5. Centralidad global agregada
global_centrality = df_centrality.groupby("country")[["frequency", "strength", "centrality_score"]].sum()
global_centrality["centrality_rank"] = global_centrality["centrality_score"].rank(ascending=False)
global_centrality = global_centrality.sort_values("centrality_score", ascending=False)
global_centrality.reset_index().to_csv(
    os.path.join(output_dir, "intermediary_summary.csv.gz"), sep=";", index=False, compression="gzip"
)

print("✅ Todos los archivos CSV han sido generados en:", output_dir)


Sección 6.3 - Agregados de dependencia ponderada

In [ ]:
# 1. Promedio ponderado de dependencia por industria
industry_avg = df.groupby("industry").apply(safe_weighted_average).reset_index(name="avg_weighted_dependency")

# 2. Top 10 dependencias ponderadas bilaterales
top_10_bilateral = weighted_dependencies.sort_values("weighted_dependency", ascending=False).head(10)

# 3. Clasificación por tramos de dependencia
df["dependency_level"] = pd.cut(
    df["dependency_value"],
    bins=[0, 0.3, 0.7, 0.9, 1],
    labels=["baja", "media", "alta", "crítica"]
)

dependency_levels_summary = df["dependency_level"].value_counts().reset_index(name="num_relaciones")


Sección 6.4 - Relaciones críticas con baja redundancia

In [ ]:
# Asumiendo que 'all_results' está disponible (output del cálculo de dependencias por industria)
from collections import defaultdict

# 1. Extraer relaciones críticas por industria
critical_relations_all = []

for industry, data in all_results.items():
    crits = identify_critical_trade_relationships(data['results'], threshold=0.7, min_paths=3)
    for c in crits:
        c["industry"] = industry
    critical_relations_all.extend(crits)

# 2. Crear dataframe consolidado
df_critical = pd.DataFrame(critical_relations_all)

# 3. Ranking de países más afectados como importadores
importadores_riesgo = df_critical.groupby("importador").size().reset_index(name="relaciones_criticas")

# 4. Ranking como exportadores
exportadores_riesgo = df_critical.groupby("exportador").size().reset_index(name="exportaciones_criticas")

# 5. Exportar si se desea
# df_critical.to_csv("relaciones_criticas.csv", index=False)


Sección 6.5 - Intermediarios críticos y centralidad

In [ ]:
# 1. Consolidar centralidad de intermediarios para todas las industrias
all_centrality = []

for industry, data in all_results.items():
    cent = data['results']['intermediary_centrality']
    for entry in cent:
        all_centrality.append({
            "industry": industry,
            "country": entry[0],
            "frequency": entry[1],
            "strength": entry[2],
            "centrality_score": entry[3]
        })

df_centrality = pd.DataFrame(all_centrality)

# 2. Top 10 intermediarios globales (acumulando por país)
global_centrality = df_centrality.groupby("country")[["frequency", "strength", "centrality_score"]].sum()
global_centrality["centrality_rank"] = global_centrality["centrality_score"].rank(ascending=False)
global_centrality = global_centrality.sort_values("centrality_score", ascending=False)

# 3. Exportar si se desea
# df_centrality.to_csv("centralidad_intermediarios_por_industria.csv", index=False)
# global_centrality.to_csv("intermediarios_globales.csv")


Visualizaciones sugeridas (iniciales en matplotlib/seaborn)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Heatmap: dependencia promedio por industria y país
pivot_df = df.pivot_table(index="dependent_country", columns="industry", values="dependency_value", aggfunc="mean")
plt.figure(figsize=(18, 10))
sns.heatmap(pivot_df, cmap="YlGnBu", center=0.5)
plt.title("Dependencia media por país e industria")
plt.tight_layout()
plt.show()

# Burbujas: Intermediarios
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=global_centrality.reset_index(),
    x="frequency", y="strength", size="centrality_score", hue="centrality_score", sizes=(100, 1000), palette="viridis"
)
plt.title("Intermediarios clave según frecuencia y fuerza")
plt.xlabel("Frecuencia como intermediario")
plt.ylabel("Fuerza acumulada")
plt.tight_layout()
plt.show()

# Dispersión: criticidad vs. caminos alternativos
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_critical, x="caminos_alternativos", y="dependencia_total", hue="criticidad", palette="coolwarm")
plt.title("Relaciones críticas: dependencia vs. redundancia")
plt.xlabel("Número de caminos alternativos")
plt.ylabel("Dependencia total")
plt.tight_layout()
plt.show()
